# Organización del número de explotaciones agrícolas

## Limpieza de los datos

In [1]:
import os

def clean_and_save_to_new_folder(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    archivos_csv = [archivo for archivo in os.listdir(input_folder) if archivo.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['agrario', 'Serie;;;2020']
    headers = "Serie;Codigo_municipio;Nombre;2020\n"

    # Limpiar y guardar los archivos CSV en la carpeta de salida
    for archivo in archivos_csv:
        archivo_original = os.path.join(input_folder, archivo)
        archivo_limpiado = os.path.join(output_folder, archivo)
        with open(archivo_original, 'r') as f_input, open(archivo_limpiado, 'w') as f_output:
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = False
                    elif '-' in line:
                        line = line.replace('-', '0')
                    if not delete:
                        f_output.write(line)
                    if 'Zarzalejo' in line:
                        delete_next = True # Eliminar filas posteriores

# Rutas de las carpetas de entrada y salida
input_folder = ".\original_data"
output_folder = ".\clean_data"

# Limpiar y guardar los archivos CSV en la nueva carpeta
clean_and_save_to_new_folder(input_folder, output_folder)


## Unión de los datos

In [17]:
import os
import csv

def unir_datos(folder_path, output_path):
    # Crear un diccionario para almacenar los datos unidos
    datos_unidos = {}

    # Obtener la lista de archivos en la carpeta
    archivos = os.listdir(folder_path)

    # Iterar sobre cada archivo en la carpeta
    campos_finales = [] # Nombre de los campos
    for archivo in archivos:
        if archivo.endswith('.csv'):
            with open(os.path.join(folder_path, archivo), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                # Saltar la primera fila que contiene los encabezados
                next(csv_reader)
                tipo = archivo.split('_')[0][:3]
                # Añadir el nombre del archivo
                if 'huertos' in archivo:
                    campos_finales.append(f'{tipo}_huertos_consumo_propio')
                elif 'tierra' in archivo:
                    campos_finales.append(f'{tipo}_tierra_pastos')
                else:
                    campos_finales.append(f'{tipo}_total_num')
                
                    
                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""
                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    key = (serie, codigo_municipio, nombre)
                    if key in datos_unidos:
                        # Si ya existe, añadir el campo 4 como un componente nuevo
                        datos_unidos[key].append(row[3])
                    else:
                        # Si no existe, crear una nueva entrada en el diccionario
                        datos_unidos[key] = [serie, codigo_municipio, nombre, row[3]]

    # Escribir los datos unidos en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        # Escribir el header
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre'] + campos_finales)
        # Escribir los datos unidos
        for key, values in datos_unidos.items():
            csv_writer.writerow(values)

# Carpeta donde se encuentran los archivos
folder_path = './clean_data'
# Ruta del archivo CSV de salida
output_path = '../union_final/explotaciones_agrarias.csv'

# Llamar a la función para unir los datos
unir_datos(folder_path, output_path)

